In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Configuración del directorio y parámetros de las imágenes
data_dir = './dataset'  # Ruta al dataset que tienes en tu PC
image_size = (224, 224)  # Tamaño de las imágenes que utilizará el modelo
batch_size = 32  # Tamaño del lote para entrenamiento
num_classes = 12  # Número de clases

In [ ]:
# Generador de datos con separación de entrenamiento y validación
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalización y división

# Conjunto de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-clase para reconocer tipo de fruta y frescura
    subset='training'
)

# Conjunto de validación
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Crear lista de nombres de clases basados en el orden de los índices
class_names = list(train_generator.class_indices.keys())
print(class_names)


In [ ]:
# Definición del modelo de red neuronal convolucional (CNN)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # 'num_classes' es el número total de categorías (fresco/pasado + tipo de fruta)
])

In [ ]:
# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenamiento del modelo
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

In [ ]:
# Evaluación del modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Loss en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")

In [ ]:
# Guardar el modelo entrenado
model.save("modelo_clasificacion_frutas.h5")